# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple of **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([73, 36, 72, 67, 58,  2, 31, 35, 32, 27, 27, 27, 20, 72, 67, 67, 57,
       35,  5, 72, 81, 18, 49, 18,  2, 17, 35, 72, 31,  2, 35, 72, 49, 49,
       35, 72, 49, 18, 70,  2, 40, 35,  2, 74,  2, 31, 57, 35, 53, 78, 36,
       72, 67, 67, 57, 35,  5, 72, 81, 18, 49, 57, 35, 18, 17, 35, 53, 78,
       36, 72, 67, 67, 57, 35, 18, 78, 35, 18, 58, 17, 35, 38, 25, 78, 27,
       25, 72, 57, 19, 27, 27, 10, 74,  2, 31, 57, 58, 36, 18, 78])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [8]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [9]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [10]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## Get the number of batches we can make
    n_batches = len(arr) // (batch_size * seq_length)
    
    ## Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size * seq_length]
    
    #Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = arr[:, n+1:n+1+seq_length]
        #foe the last batch, an extra step is needed
        if x.size != y.size:
            #y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
            y = np.append(y, arr[:,0].reshape((-1,1)), axis=1)
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [11]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [12]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[73 36 72 67 58  2 31 35 32 27]
 [17 38 78 35 58 36 72 58 35 72]
 [ 2 78 29 35 38 31 35 72 35  5]
 [17 35 58 36  2 35 82 36 18  2]
 [35 17 72 25 35 36  2 31 35 58]
 [82 53 17 17 18 38 78 35 72 78]
 [35 55 78 78 72 35 36 72 29 35]
 [13 30 49 38 78 17 70 57 19 35]]

y
 [[36 72 67 58  2 31 35 32 27 27]
 [38 78 35 58 36 72 58 35 72 58]
 [78 29 35 38 31 35 72 35  5 38]
 [35 58 36  2 35 82 36 18  2  5]
 [17 72 25 35 36  2 31 35 58  2]
 [53 17 17 18 38 78 35 72 78 29]
 [55 78 78 72 35 36 72 29 35 17]
 [30 49 38 78 17 70 57 19 35 66]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [13]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [14]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers,dropout = drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.reshape(-1, self.n_hidden)
        # return the final output and the hidden state
        #put x through the fully-connected layer
        out = self.fc(out)
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Now we can actually train the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.



In [16]:
#Define the model
#number of hidden nodes in LSTM layers, number of layers, drop probability, learning rate
n_hidden = 375
n_layers=2
drop_prob=0.5
lr=0.001
#batch size (how many batches) and length of sequence
batch_size = 128
n_seq = 100
#how big each input vector is
size_input = len(chars)
#number of epochs
epochs = 50
#model name to save
model_name = 'rnn_x_epoch.net'
#Clipping gradients to prevent exploding gradient values
clip=5

we'll create the network itself, with some given hyperparameters. 

In [17]:

#Define the model. Tokens is basically the set of characters
model = CharRNN(tokens=chars, n_hidden=n_hidden, n_layers=n_layers, drop_prob=drop_prob, lr=lr)
print(model)
#move the model to gpu
if train_on_gpu:
    model.cuda()
    print("Model is on GPU")
else:
    print("No GPU is available")

#count parameters
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print("There are",params,"parameters in the model")
print("There are",len(encoded), "characters in the text")

CharRNN(
  (lstm): LSTM(83, 375, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=375, out_features=83, bias=True)
)
Model is on GPU
There are 1849208 parameters in the model
There are 1985223 characters in the text


Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients

In [18]:
#Define the optimizer and the loss criterion
optim = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [19]:
#create validation data
val_idx = int(len(encoded)*0.9)
data, val_data = encoded[:val_idx], encoded[val_idx:]

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

In [28]:
#Training Model
val_loss_min = np.Inf
for e in range(epochs):
    train_loss = 0
    train_losses = []
    #initialize the hidden and cell units
    hidden = model.init_hidden(batch_size=batch_size)
    model.train()

    for x, y in get_batches(data, batch_size=batch_size, seq_length=n_seq):
        count += 1

        x = torch.from_numpy(one_hot_encode(x, n_labels=size_input))
        #apparently, one hot encoding of y is not necessary
        #y = torch.from_numpy(one_hot_encode(y, n_labels=size_input)).reshape((-1, size_input))
        y = torch.from_numpy(y)

        if train_on_gpu:
            x, y = x.cuda(), y.cuda()
        #hidden data is transformed so that the model does not backpropagate through the hidden values
        #this happens because hidden value is an output of the forward method within the model. Weights and outputs are 
        #backpropagated essentially
        hidden = tuple(h.data for h in hidden)
        #reset gradients. This resets gradients after every sequence of x and y
        optim.zero_grad()

        output, hidden = model(x, hidden)
        #y needs to be converted into long integer format after flattening applied to it
        loss = criterion(output, y.reshape(batch_size*n_seq).long())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)

        optim.step()

        train_loss = loss.item()
        train_losses.append(train_loss)


    #MODEL VALIDATION
    val_hidden = model.init_hidden(batch_size=batch_size)
    val_losses =[]
    model.eval()

    for x,y in get_batches(val_data, batch_size, n_seq):

        x = torch.from_numpy(one_hot_encode(x, n_labels=size_input))
        #apparently, one hot encoding of y is not necessary
        #y = torch.from_numpy(one_hot_encode(y, n_labels=size_input)).reshape((-1, size_input))
        y = torch.from_numpy(y)

        if train_on_gpu:
            x, y = x.cuda(), y.cuda()

        val_hidden = tuple(h.data for h in hidden)
        
        #reset gradients
        optim.zero_grad()

        output, hidden = model(x, val_hidden)
        #y needs to be converted into long integer format after flattening applied to it
        loss = criterion(output, y.reshape(batch_size*n_seq).long())

        val_loss = loss.item()
        val_losses.append(val_loss)
        #if eval_count % 10 == 0:
        #    print(eval_count)
    av_val_loss_ep = np.mean(np.array(val_losses))

    print("Epoch: {}/{}...".format(e+1, epochs),
            "Loss: {:.4f}...".format(loss.item()),
            "Val Loss: {:.4f}".format(av_val_loss_ep))
    
    #saving the model if loss is less than the last
    if av_val_loss_ep <= val_loss_min:

        print('Validation loss decreased ({:.4f} --> {:.4f}). Saving Model...'.format(
            val_loss_min, av_val_loss_ep))

        checkpoint = {'n_hidden': model.n_hidden,
                'n_layers': model.n_layers,
                'state_dict': model.state_dict(),
                'tokens': model.chars}
        with open(model_name, 'wb') as f:
            torch.save(checkpoint, f)

        val_loss_min = av_val_loss_ep


Epoch: 1/50... Loss: 2.8340... Val Loss: 2.8124
Validation loss decreased (inf --> 2.8124). Saving Model...
Epoch: 2/50... Loss: 2.3203... Val Loss: 2.2863
Validation loss decreased (2.8124 --> 2.2863). Saving Model...
Epoch: 3/50... Loss: 2.1200... Val Loss: 2.0776
Validation loss decreased (2.2863 --> 2.0776). Saving Model...
Epoch: 4/50... Loss: 1.9823... Val Loss: 1.9309
Validation loss decreased (2.0776 --> 1.9309). Saving Model...
Epoch: 5/50... Loss: 1.8734... Val Loss: 1.8125
Validation loss decreased (1.9309 --> 1.8125). Saving Model...
Epoch: 6/50... Loss: 1.8012... Val Loss: 1.7262
Validation loss decreased (1.8125 --> 1.7262). Saving Model...
Epoch: 7/50... Loss: 1.7373... Val Loss: 1.6589
Validation loss decreased (1.7262 --> 1.6589). Saving Model...
Epoch: 8/50... Loss: 1.6933... Val Loss: 1.6096
Validation loss decreased (1.6589 --> 1.6096). Saving Model...
Epoch: 9/50... Loss: 1.6409... Val Loss: 1.5609
Validation loss decreased (1.6096 --> 1.5609). Saving Model...
Epoc

## Checkpoint

Last part of the model checks if the validation error is improved and saves the moedl. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

    if av_val_loss_ep <= val_loss_min:

        print('Validation loss decreased ({:.4f} --> {:.4f}). Saving Model...'.format(
            val_loss_min, av_val_loss_ep))

        checkpoint = {'n_hidden': model.n_hidden,
                'n_layers': model.n_layers,
                'state_dict': model.state_dict(),
                'tokens': model.chars}
        with open(model_name, 'wb') as f:
            torch.save(checkpoint, f)

        val_loss_min = av_val_loss_ep

Now loading the best performing model:

In [20]:
# Here we have loaded in a model that trained 
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
model = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [21]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [22]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [23]:
print(sample(model, 1000, prime='Anna', top_k=5))

Anna Agafea Mihalovna, taking
his eye, all she was dead at the mercy of his strength of secretary and
the party, they had no one of the conversation at the carriage and had been for her
heart in the carriage of his hands to set out them in her, and was
so stopped of this to her to her, son to do.

All his feeling was several times whose were so much to be thinking,
and when this course, had so that his brother had to be the princess; but to have
seemed all the cheekful and a prince, that he had not been fascinating her since, and his
brother and had been beginning to talk to the conversation with the
down of the sets of angry for the pillow; to the point of coarse, but
the more and so that they held as something with his brother and handsome that he
did not like some pictures to
her at the children there, he had not been temition.

The murder women to his heart a subject.

"What's so the sounds of such a principle, who were you so many times,"
said Karanovitch, "since I should have bee

## Loading a checkpoint

In [24]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [25]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said that he had
not seen her, and thrist her shall sigh feeling.

"You were not so much in the disagage the time which is
such thoughts!" he said; "I don't see her in a country wife, but I didn't
truth, and here as I have at the carpet of a man of the points in the
position. To this me that it was in his heart to her son, that he's an expression. If
you're not thinking," she said, and stowing his side.

"I suffer," answered the princess, and the same sister in the stat of
and wolding in his face. And the same side he had never to be
asked to him to tell a secretary that he would have seemed to him
by that secretary of the drawing room.

"Alexey Alexandrovitch's a sound of tempour. But that's a discome, and
I will get into the same momon, as he counted the prayer of servant, and
that's a little from the principle, and I can't be thinking to
me to that me. The marshal of all the dinner. And what's not the same
means," said Levin, but that they were so much straight and
seeing 